<a href="https://colab.research.google.com/github/Muhammad-BILAL-5/Reflection-Agents/blob/main/reflection_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install --quiet --upgrade langchain langchain-google-genai langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.3/145.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.2 MB/s eta 0:00:00


In [1]:
from google.colab import userdata
import os

os.environ["LANGCHAIN_API_KEY"] = userdata.get("langchai_api_key")
os.environ["GOOGLE_API_KEY"] = userdata.get("Gemini_Api_Key")

In [6]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_google_genai import ChatGoogleGenerativeAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

generate = prompt | llm
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

Antoine de Saint-Exupéry’s *The Little Prince*, though seemingly a simple children’s story, carries profound messages that resonate deeply with the complexities of modern childhood.  While the world has changed drastically since the book’s publication, the core struggles of childhood, the pressures of conformity, and the importance of imagination remain timeless themes that make *The Little Prince* as relevant today as ever.  The story gently reminds us of the essential values often lost in a world increasingly dominated by technology and a focus on material success.

One key aspect of the book’s enduring relevance lies in its exploration of imagination and wonder.  In a world saturated with structured activities and digital entertainment, children are often pressured to conform to prescribed roles and expectations. *The Little Prince*, with its whimsical characters and fantastical planets, champions the power of creativity and the importance of nurturing individual perspectives.  The 

In [7]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

This is a good starting point for your essay on the relevance of *The Little Prince* in modern childhood. It touches on important themes and provides a decent overview of the book's message. However, there's room for improvement in terms of depth of analysis, specific examples, and overall structure.

**Critique and Recommendations:**

* **Length and Depth:** The essay is a bit general. While you mention important themes, you could delve deeper into each one. Aim to expand the essay by providing concrete examples from the book to support your claims.  For instance, when discussing imagination, you could analyze specific scenes like the Prince's encounter with the pilot drawing a boa constrictor or his creation of the sheep.  A good target length would be around 500-750 words, depending on the assignment requirements.

* **Analysis of Modern Childhood:** While you mention "modern childhood," you could be more specific about the challenges children face today.  Consider discussing the im

In [8]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

Antoine de Saint-Exupéry’s *The Little Prince*, a seemingly simple tale of a pilot stranded in the desert and his encounter with a curious boy from another planet, offers a timeless reflection on the human condition, one that resonates with particular poignancy in the context of modern childhood.  While the world has undergone a dramatic transformation since the book’s publication, the core challenges of childhood, the allure of conformity, and the vital importance of nurturing imagination remain strikingly relevant.  *The Little Prince* serves as a gentle yet powerful reminder of the essential values often obscured by the relentless pace and technological saturation of contemporary life.

One of the book’s most enduring strengths lies in its celebration of imagination and wonder.  In a world where children’s lives are increasingly structured by scheduled activities, standardized testing, and the pervasive influence of digital entertainment, the Prince’s unconstrained creativity stands

In [11]:
from typing import Annotated, List, Sequence
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]


async def generation_node(state: State) -> State:
    return {"messages": [await generate.ainvoke(state["messages"])]}


async def reflection_node(state: State) -> State:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [state["messages"][0]] + [
        cls_map[msg.type](content=msg.content) for msg in state["messages"][1:]
    ]
    res = await reflect.ainvoke(translated)
    # We treat the output of this as human feedback for the generator
    return {"messages": [HumanMessage(content=res.content)]}


builder = StateGraph(State)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_edge(START, "generate")


def should_continue(state: State):
    if len(state["messages"]) > 3:
        # End after 3 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "1"}}
async for event in graph.astream(
    {
        "messages": [
            HumanMessage(
                content="Generate an essay on the topicality of The Little Prince and its message in modern life"
            )
        ],
    },
    config,
):
    print(event)
    print("---")

{'generate': {'messages': [AIMessage(content="Antoine de Saint-Exupéry's *The Little Prince*, though seemingly a simple children's story, continues to resonate deeply with readers of all ages, proving its timeless relevance.  Its exploration of essential human values like love, responsibility, and the importance of imagination transcends generational gaps, offering a poignant commentary on the pitfalls of modern life. While our world has changed drastically since the book's publication in 1943, the core message about prioritizing meaningful connections and nurturing our inner child remains strikingly topical.\n\nOne of the most prominent themes in *The Little Prince* is the critique of adult preoccupation with superficial matters. The prince encounters adults obsessed with numbers, power, and appearances, neglecting the truly important aspects of life. This resonates powerfully in our contemporary society, often driven by consumerism, social status, and quantifiable achievements. The p